In [1]:
%matplotlib inline

In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import helper_functions as hp

from pathlib import Path

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import rescale

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Natural Images

https://www.kaggle.com/datasets/prasunroy/natural-images

### Read file names and classes

In [3]:
DESKTOP = Path().home() / 'Desktop'

In [4]:
nature_dir = DESKTOP / 'natural_images'

In [19]:
def extract_images_and_labels(directory):
    all_images = []
    all_labels = []
    
    for folder in directory.iterdir():
        if not folder.is_dir():
            continue
        images = [str(path) for path in folder.glob('*')]
        all_images.extend(images)
        all_labels.extend([folder.name] * len(images))
        
    return all_images, all_labels

In [20]:
all_images, all_labels = extract_images_and_labels(nature_dir)

print('# images: ', len(all_images))
print('# labels: ', len(all_labels))
print(all_images[0])
print(all_labels[0])

# images:  6899
# labels:  6899
C:\Users\a1056968\Desktop\natural_images\airplane\airplane_0000.jpg
airplane


In [21]:
labels_dict = {el.name: i for i, el in enumerate(nature_dir.iterdir())}
labels_dict

{'airplane': 0,
 'car': 1,
 'cat': 2,
 'dog': 3,
 'flower': 4,
 'fruit': 5,
 'motorbike': 6,
 'person': 7}

In [22]:
all_label_ids = [labels_dict[l] for l in all_labels]

In [23]:
pd.DataFrame({
    'image_path': all_images,
    'labels': all_labels,
    'label_ids': all_label_ids,
}).sample(5)

,image_path,labels,label_ids
5866,C:\Users\a1056968\Desktop\natural_images\motor...,motorbike,6
2757,C:\Users\a1056968\Desktop\natural_images\dog\d...,dog,3
4463,C:\Users\a1056968\Desktop\natural_images\fruit...,fruit,5
3283,C:\Users\a1056968\Desktop\natural_images\flowe...,flower,4
2211,C:\Users\a1056968\Desktop\natural_images\cat\c...,cat,2


### Load images using `ImageDataGenerator()`

In [24]:
generator = ImageDataGenerator()

In [25]:
images= generator.flow_from_directory(nature_dir)

Found 6899 images belonging to 8 classes.


In [26]:
next_batch = images.__next__()

### Pipeline using `Dataset()`

#### Example 1

In [29]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [30]:
for el in dataset:
    print(type(el))
    print(el.numpy())

<class 'tensorflow.python.framework.ops.EagerTensor'>
1
<class 'tensorflow.python.framework.ops.EagerTensor'>
2
<class 'tensorflow.python.framework.ops.EagerTensor'>
3


#### Example 2

In [40]:
dataset = tf.data.Dataset.from_tensor_slices(all_images)

In [43]:
for el in dataset.take(5):
    print(el)

tf.Tensor(b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0000.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0001.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0002.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0003.jpg', shape=(), dtype=string)
tf.Tensor(b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0004.jpg', shape=(), dtype=string)


#### Initial

In [48]:
dataset = tf.data.Dataset.from_tensor_slices((all_images, all_label_ids))

In [51]:
for img, l in dataset.take(2):
    print(img.numpy(), l.numpy())

b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0000.jpg' 0
b'C:\\Users\\a1056968\\Desktop\\natural_images\\airplane\\airplane_0001.jpg' 0


#### Map

#### Heading

In [56]:
def read_image(filename, label):
    return (filename, label)

In [173]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .map(read_image)

In [142]:
for img, l in dataset.take(2):
    print(img.numpy(), l.numpy())

[[[0.87058824 0.9372549  0.8980392 ]
  [0.87159926 0.9382659  0.89905024]
  [0.8745098  0.9411765  0.9019608 ]
  ...
  [0.8        0.8509804  0.8235294 ]
  [0.80291057 0.85389096 0.82644   ]
  [0.79675245 0.84773284 0.82028186]]

 [[0.8701593  0.936826   0.8976103 ]
  [0.87117034 0.937837   0.8986213 ]
  [0.8740809  0.94074756 0.9015319 ]
  ...
  [0.80006033 0.85134566 0.8244476 ]
  [0.80206615 0.85326767 0.8258167 ]
  [0.7962498  0.8480143  0.8205633 ]]

 [[0.86856616 0.9352328  0.89601713]
  [0.86957717 0.93624383 0.89702815]
  [0.8724877  0.9391544  0.8999387 ]
  ...
  [0.8002844  0.8527023  0.8278579 ]
  [0.79892963 0.8509526  0.82350165]
  [0.79438287 0.8490598  0.82160884]]

 ...

 [[0.5217735  0.44334215 0.3492245 ]
  [0.47206074 0.39565143 0.30355585]
  [0.40148017 0.33089194 0.24461742]
  ...
  [0.5336397  0.4356005  0.34932598]
  [0.5398294  0.44179016 0.35551566]
  [0.535488   0.43744877 0.35117427]]

 [[0.5307847  0.45235333 0.3582357 ]
  [0.4736041  0.3971948  0.30509922]


#### Read file

In [143]:
def read_image(filename, label):
    image = tf.io.read_file(filename)
    return (image, label)

In [144]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .map(read_image)

In [145]:
for img, l in dataset.take(1):
    print(img.numpy(), l.numpy())
    pass

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x00h\x01,\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x9

#### Decode bytes

In [146]:
def read_image(filename, label):
    image_bytes = tf.io.read_file(filename)
    image_data = tf.image.decode_jpeg(image_bytes)
    return (image_data, label)

In [147]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .map(read_image)

In [148]:
for img, l in dataset.take(1):
    print(img.numpy(), l.numpy())

[[[222 239 229]
  [222 239 229]
  [223 240 230]
  ...
  [204 217 210]
  [205 218 211]
  [203 216 209]]

 [[221 238 228]
  [221 238 228]
  [222 239 229]
  ...
  [205 220 213]
  [202 215 208]
  [202 217 210]]

 [[221 238 228]
  [221 238 228]
  [222 239 229]
  ...
  [197 214 208]
  [202 217 210]
  [201 218 210]]

 ...

 [[148 128 104]
  [147 127 103]
  [145 127 103]
  ...
  [124  99  77]
  [124  99  77]
  [123  98  76]]

 [[131 111  87]
  [123 103  79]
  [111  93  71]
  ...
  [135 110  88]
  [135 110  88]
  [134 109  87]]

 [[137 117  93]
  [125 105  81]
  [109  91  69]
  ...
  [139 114  92]
  [141 116  94]
  [139 114  92]]] 0


#### Scale

In [149]:
def read_image(filename, label):
    image_bytes = tf.io.read_file(filename)
    image_data = tf.image.decode_jpeg(image_bytes)
    image_scaled = tf.cast(image_data, float) / 255.0
    
    return (image_scaled, label)

In [150]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .map(read_image)

In [151]:
for img, l in dataset.take(1):
    print(img.numpy().shape, l.numpy().shape)

(104, 300, 3) ()


#### Resize

In [152]:
IMAGE_SIZE = (256, 256)
IMAGE_SIZE_WITH_CHANNELS = (256, 256, 3)

In [153]:
def read_image(filename, label):
    image_bytes = tf.io.read_file(filename)
    image_data = tf.image.decode_jpeg(image_bytes)
    image_scaled = tf.cast(image_data, float) / 255.0
    image_resized = tf.image.resize(image_scaled, IMAGE_SIZE)
    
    return (image_resized, label)

In [154]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .map(read_image) \
    .batch(4)

In [155]:
for img, l in dataset.take(1):
    print(img.numpy().shape, l.numpy().shape)

(4, 256, 256, 3) (4,)


#### Batch

In [156]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .shuffle(len(all_images)) \
    .map(read_image) \
    .batch(4)

In [157]:
for img, l in dataset.take(1):
    print(img.numpy().shape, l.numpy().shape)

(4, 256, 256, 3) (4,)


#### Model

In [163]:
n_labels = len(labels_dict)

model = Sequential([
    Input(shape=IMAGE_SIZE_WITH_CHANNELS),
    
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPool2D(),
    Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPool2D(),
    Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'),
    Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu'),
    MaxPool2D(),
    
    Dense(32, activation='relu'),
    Dense(n_labels, activation = 'softmax'),
])

In [164]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 256, 256, 32)      896       
                                                                 
 conv2d_19 (Conv2D)          (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 128, 128, 16)      4624      
                                                                 
 conv2d_21 (Conv2D)          (None, 128, 128, 16)      2320      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                 

#### Repeat

In [176]:
dataset = tf.data.Dataset \
    .from_tensor_slices((all_images, all_label_ids)) \
    .shuffle(len(all_images)) \
    .map(read_image) \
    .batch(4) \
    .repeat()

#### Compile and train

In [177]:
dataset.element_spec

(TensorSpec(shape=(None, 256, 256, None), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [171]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam')

In [180]:
# TODO
# model.fit(dataset, epochs=10, steps_per_epoch=len(all_images) // 4)